<a href="https://colab.research.google.com/github/lonnieljyu/advent_of_code_2019/blob/day3/day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

input_path = '/content/drive/My Drive/Advent of Code/2019/day3.txt'

In [0]:
from typing import Tuple

def get_new_cursor(
    cursor: Tuple[(int, int)], direction = str) -> Tuple[(int, int)]:
    x_offset = y_offset = 0
    if direction == 'R':
        x_offset = 1
    elif direction == 'L':
        x_offset = -1
    elif direction == 'U':
        y_offset = 1
    elif direction == 'D':
        y_offset = -1
    return cursor[0] + x_offset, cursor[1] + y_offset


with open(input_path) as file:
    lines = file.readlines()

origin = cursor = (0, 0)
path_coordinates = set()
for segment in lines[0].split(','):
    for _ in range(int(segment[1:])):
        cursor = get_new_cursor(cursor, segment[0])
        path_coordinates.add(cursor)


cursor = origin
min_distance = float('inf')
for segment in lines[1].split(','):
    for _ in range(int(segment[1:])):
        cursor = get_new_cursor(cursor, segment[0])
        if cursor in path_coordinates:
            distance = abs(cursor[0]) + abs(cursor[1])
            if distance < min_distance:
                min_distance = distance
print(min_distance)

In [0]:
with open(input_path) as file:
    lines = file.readlines()

origin = cursor = (0, 0)
path_dict = dict()
steps = 0
segments = lines[0].split(',')
for segment in lines[0].split(','):
    for _ in range(int(segment[1:])):
        cursor = get_new_cursor(cursor, segment[0])
        steps += 1
        if cursor not in path_dict:
            path_dict[cursor] = steps


cursor = origin
min_steps = float('inf')
steps = 0
for segment in lines[1].split(','):
    for _ in range(int(segment[1:])):
        cursor = get_new_cursor(cursor, segment[0])
        steps += 1
        if cursor in path_dict:
            new_steps = steps + path_dict[cursor]
            if new_steps < min_steps:
                min_steps = new_steps
print(min_steps)